In [2]:
# le = life expectancy
# d = drinks
# ds = dataset

import pandas as pd
import numpy as np

# load both datasets into pandas dataframes
d_raw = pd.read_csv('drinks.csv')
le_raw = pd.read_csv('life_expectancy.csv')

# make a copy of each dataframe and clean data to prepare for merge
d = d_raw.copy()
le = le_raw.copy()
d.drop('index', axis=1, inplace=True)
le.drop('index', axis=1, inplace=True)

# fix discrepencies le['CountryDisplay'] and d['country'] columns before merging on them
missmatched_countries = le[~le['CountryDisplay'].isin(d['country'])]
print(missmatched_countries['CountryDisplay'].unique())

country_map = {
    'Bolivia (Plurinational State of)' : 'Bolivia', 
    'Saint Lucia' : 'St. Lucia',
    'Syrian Arab Republic' : 'Syria',
     'Sao Tome and Principe' : 'Sao Tome & Principe',
    'Republic of Korea' : 'South Korea',
    'Antigua and Barbuda' : 'Antigua & Barbuda',
    'Saint Vincent and the Grenadines' : 'St. Vincent & the Grenadines',
    'Saint Kitts and Nevis' : 'St. Kitts & Nevis',
    'United Republic of Tanzania' : 'Tanzania',
    'The former Yugoslav republic of Macedonia' : 'Macedonia',
    'South Sudan' : 'Sudan',
    'Micronesia (Federated States of)' : 'Micronesia',
    'Brunei Darussalam' : 'Brunei',
    'United Kingdom of Great Britain and Northern Ireland' : 'United Kingdom',
    "Côte d'Ivoire" : "Cote d'Ivoire",
    'United States of America' : 'USA',
    "Democratic People's Republic of Korea" : "North Korea",
    'Viet Nam' : 'Vietnam',
    'Democratic Republic of the Congo' : 'DR Congo',
    'Republic of Moldova' : 'Moldova',
    'Iran (Islamic Republic of)' : 'Iran',
    'Bosnia and Herzegovina' : 'Bosnia-Herzegovina',
    'Timor_Leste' : 'Timor-Leste',
    'Venezuela (Bolivarian Republic of)' : 'Venezuela',
    "Lao People's Democratic Republic" : "Laos",
    'Trinidad and Tobago' : 'Trinidad & Tobago',
    'Guinea_Bissau' : 'Guinea-Bissau'
}
le['CountryDisplay'] = le['CountryDisplay'].replace(country_map)

missmatched_countries = le[~le['CountryDisplay'].isin(d['country'])]
print(missmatched_countries['CountryDisplay'].unique()) # should now be empty

ds = pd.merge(le, d, left_on='CountryDisplay', right_on='country', how='inner')
print(f'le shape: {le.shape} ds shape: {ds.shape}')

# update life expectancy numbers to be consistent ('WHOSIS_000015' was counting years AFTER 60).
ds['Numeric'] = np.where(
    ds['GhoCode'] == 'WHOSIS_000015',
    ds['Numeric'].astype(float) + 60,
    ds['Numeric'].astype(float)
)
ds['DisplayValue'] = ds['Numeric'].astype(int).round(0)
ds.head()

['Bolivia (Plurinational State of)' 'Saint Lucia' 'Syrian Arab Republic'
 'Sao Tome and Principe' 'Republic of Korea' 'Antigua and Barbuda'
 'Saint Vincent and the Grenadines' 'Saint Kitts and Nevis'
 'United Republic of Tanzania' 'The former Yugoslav republic of Macedonia'
 'South Sudan' 'Micronesia (Federated States of)' 'Brunei Darussalam'
 'United Kingdom of Great Britain and Northern Ireland' "Côte d'Ivoire"
 'United States of America' "Democratic People's Republic of Korea"
 'Viet Nam' 'Democratic Republic of the Congo' 'Republic of Moldova'
 'Iran (Islamic Republic of)' 'Bosnia and Herzegovina' 'Timor_Leste'
 'Venezuela (Bolivarian Republic of)' "Lao People's Democratic Republic"
 'Trinidad and Tobago' 'Guinea_Bissau']
[]
le shape: (6408, 16) ds shape: (6408, 21)


,GhoCode,GhoDisplay,PublishStateCode,PublishStateDisplay,YearCode,YearDisplay,RegionCode,RegionDisplay,WorldBankIncomeGroupGroupCode,WorldBankIncomeGroupDisplay,...,CountryDisplay,SexCode,SexDisplay,DisplayValue,Numeric,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,1990,1990,AMR,Americas,WB_LMI,Lower_middle_income,...,Nicaragua,MLE,Male,68,68.0,Nicaragua,78,118,1,3.5
1,WHOSIS_000002,Healthy life expectancy (HALE) at birth (years),PUBLISHED,Published,2012,2012,EUR,Europe,WB_HI,High_income,...,Ireland,MLE,Male,69,69.3,Ireland,313,118,165,11.4
2,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,2000,2000,EMR,Eastern Mediterranean,WB_LI,Low_income,...,Yemen,BTSX,Both sexes,61,61.0,Yemen,6,0,0,0.1
3,WHOSIS_000001,Life expectancy at birth (years),PUBLISHED,Published,2000,2000,AFR,Africa,WB_LMI,Lower_middle_income,...,Nigeria,FMLE,Female,48,48.0,Nigeria,42,5,2,9.1
4,WHOSIS_000015,Life expectancy at age 60 (years),PUBLISHED,Published,1990,1990,SEAR,South_East Asia,WB_LMI,Lower_middle_income,...,Thailand,FMLE,Female,79,79.0,Thailand,99,258,1,6.4


In [4]:
import plotly.express as px
ds_country_sort = ds.copy().sort_values('CountryDisplay', ascending=True)
fig = px.box(ds_country_sort, x='CountryDisplay', y='DisplayValue', title='Life Expectancy by Country (A-Z)', labels={'CountryDisplay': 'Country', 'DisplayValue': 'Life Expectancy'})
fig.update_layout(xaxis_tickangle=45, width=1300, height=700)
fig.show()